In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import datetime

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
# テストデータ
#train = pd.read_csv('../input/devtrain/devtrain.csv')
#test = pd.read_csv('../input/devtestv2/devtest.csv')
#train = pd.read_csv('devtrain.csv')
#test = pd.read_csv('devtest.csv')

train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [3]:
test.head()

,id,date,country,store,product
0,136950,2022-01-01,Argentina,Kaggle Learn,Using LLMs to Improve Your Coding
1,136951,2022-01-01,Argentina,Kaggle Learn,Using LLMs to Train More LLMs
2,136952,2022-01-01,Argentina,Kaggle Learn,Using LLMs to Win Friends and Influence People
3,136953,2022-01-01,Argentina,Kaggle Learn,Using LLMs to Win More Kaggle Competitions
4,136954,2022-01-01,Argentina,Kaggle Learn,Using LLMs to Write Better


In [4]:
train.head()

,id,date,country,store,product,num_sold
0,0,2017-01-01,Argentina,Kaggle Learn,Using LLMs to Improve Your Coding,63
1,1,2017-01-01,Argentina,Kaggle Learn,Using LLMs to Train More LLMs,66
2,2,2017-01-01,Argentina,Kaggle Learn,Using LLMs to Win Friends and Influence People,9
3,3,2017-01-01,Argentina,Kaggle Learn,Using LLMs to Win More Kaggle Competitions,59
4,4,2017-01-01,Argentina,Kaggle Learn,Using LLMs to Write Better,49


In [5]:
# trainとtestを縦結合
data = pd.concat([train, test], sort=False)
data.head()

,id,date,country,store,product,num_sold
0,0,2017-01-01,Argentina,Kaggle Learn,Using LLMs to Improve Your Coding,63.0
1,1,2017-01-01,Argentina,Kaggle Learn,Using LLMs to Train More LLMs,66.0
2,2,2017-01-01,Argentina,Kaggle Learn,Using LLMs to Win Friends and Influence People,9.0
3,3,2017-01-01,Argentina,Kaggle Learn,Using LLMs to Win More Kaggle Competitions,59.0
4,4,2017-01-01,Argentina,Kaggle Learn,Using LLMs to Write Better,49.0


# Country,Store,Productの変換

In [6]:
#country
country = data['country']
# ダミー変数に変換する
# ダミー変数：カテゴリーデータを0と1だけで構成されているカテゴリーごとの列データに変換したもの
data = pd.get_dummies(data, columns=['country'])
data['country'] = country

#store
store = data['store']
# ダミー変数に変換する
# ダミー変数：カテゴリーデータを0と1だけで構成されているカテゴリーごとの列データに変換したもの
data = pd.get_dummies(data, columns=['store'])
data['store'] = store

#Product
product = data['product']
# ダミー変数に変換する
# ダミー変数：カテゴリーデータを0と1だけで構成されているカテゴリーごとの列データに変換したもの
data = pd.get_dummies(data, columns=['product'])
data['product'] = product

data.head()

,id,date,num_sold,country_Argentina,country_Canada,country_Estonia,country_Japan,country_Spain,country,store_Kagglazon,store_Kaggle Learn,store_Kaggle Store,store,product_Using LLMs to Improve Your Coding,product_Using LLMs to Train More LLMs,product_Using LLMs to Win Friends and Influence People,product_Using LLMs to Win More Kaggle Competitions,product_Using LLMs to Write Better,product
0,0,2017-01-01,63.0,1,0,0,0,0,Argentina,0,1,0,Kaggle Learn,1,0,0,0,0,Using LLMs to Improve Your Coding
1,1,2017-01-01,66.0,1,0,0,0,0,Argentina,0,1,0,Kaggle Learn,0,1,0,0,0,Using LLMs to Train More LLMs
2,2,2017-01-01,9.0,1,0,0,0,0,Argentina,0,1,0,Kaggle Learn,0,0,1,0,0,Using LLMs to Win Friends and Influence People
3,3,2017-01-01,59.0,1,0,0,0,0,Argentina,0,1,0,Kaggle Learn,0,0,0,1,0,Using LLMs to Win More Kaggle Competitions
4,4,2017-01-01,49.0,1,0,0,0,0,Argentina,0,1,0,Kaggle Learn,0,0,0,0,1,Using LLMs to Write Better


# 季節性データの作成

In [7]:
data['date'] = pd.to_datetime(data['date'])
# PandasでSeries.dt()による日付変換
# https://qiita.com/Takemura-T/items/79b16313e45576bb6492
data['dayofweek'] = data['date'].dt.dayofweek
# 以下警告文が出ている
# FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated. Please use Series.dt.isocalendar().week instead.
data['weekofyear'] = data['date'].dt.weekofyear
data['quarter'] = data['date'].dt.quarter

data['monday'] = data.date.dt.weekday.eq(0).astype(np.uint8)
data['tuesday'] = data.date.dt.weekday.eq(1).astype(np.uint8)
data['wednesday'] = data.date.dt.weekday.eq(2).astype(np.uint8)
data['thursday'] = data.date.dt.weekday.eq(3).astype(np.uint8)
data['friday'] = data.date.dt.weekday.eq(4).astype(np.uint8)
data['saturday'] = data.date.dt.weekday.eq(5).astype(np.uint8)
data['sunday'] = data.date.dt.weekday.eq(6).astype(np.uint8)

data['is_month_end'] = data['date'].dt.is_month_end
data['is_month_start'] = data['date'].dt.is_month_start
data['is_quarter_end'] = data['date'].dt.is_quarter_end
data['is_quarter_start'] = data['date'].dt.is_quarter_start
data['is_year_end'] = data['date'].dt.is_year_end
data['is_year_start'] = data['date'].dt.is_year_start

data.head()

,id,date,num_sold,country_Argentina,country_Canada,country_Estonia,country_Japan,country_Spain,country,store_Kagglazon,...,thursday,friday,saturday,sunday,is_month_end,is_month_start,is_quarter_end,is_quarter_start,is_year_end,is_year_start
0,0,2017-01-01,63.0,1,0,0,0,0,Argentina,0,...,0,0,0,1,False,True,False,True,False,True
1,1,2017-01-01,66.0,1,0,0,0,0,Argentina,0,...,0,0,0,1,False,True,False,True,False,True
2,2,2017-01-01,9.0,1,0,0,0,0,Argentina,0,...,0,0,0,1,False,True,False,True,False,True
3,3,2017-01-01,59.0,1,0,0,0,0,Argentina,0,...,0,0,0,1,False,True,False,True,False,True
4,4,2017-01-01,49.0,1,0,0,0,0,Argentina,0,...,0,0,0,1,False,True,False,True,False,True


## 訓練データ、テストデータ作成

In [8]:
train = data[:len(train)]
test = data[len(train):]

# 国別モデル作成

In [9]:
train_Argentina = train[train['country_Argentina'] == 1]
train_Canada = train[train['country_Canada'] == 1]
train_Estonia = train[train['country_Estonia'] == 1]
train_Japan = train[train['country_Japan'] == 1]
train_Spain = train[train['country_Spain'] == 1]


In [10]:
train_Argentina

,id,date,num_sold,country_Argentina,country_Canada,country_Estonia,country_Japan,country_Spain,country,store_Kagglazon,...,thursday,friday,saturday,sunday,is_month_end,is_month_start,is_quarter_end,is_quarter_start,is_year_end,is_year_start
0,0,2017-01-01,63.0,1,0,0,0,0,Argentina,0,...,0,0,0,1,False,True,False,True,False,True
1,1,2017-01-01,66.0,1,0,0,0,0,Argentina,0,...,0,0,0,1,False,True,False,True,False,True
2,2,2017-01-01,9.0,1,0,0,0,0,Argentina,0,...,0,0,0,1,False,True,False,True,False,True
3,3,2017-01-01,59.0,1,0,0,0,0,Argentina,0,...,0,0,0,1,False,True,False,True,False,True
4,4,2017-01-01,49.0,1,0,0,0,0,Argentina,0,...,0,0,0,1,False,True,False,True,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
136885,136885,2021-12-31,251.0,1,0,0,0,0,Argentina,1,...,0,1,0,0,True,False,True,False,True,False
136886,136886,2021-12-31,252.0,1,0,0,0,0,Argentina,1,...,0,1,0,0,True,False,True,False,True,False
136887,136887,2021-12-31,39.0,1,0,0,0,0,Argentina,1,...,0,1,0,0,True,False,True,False,True,False
136888,136888,2021-12-31,233.0,1,0,0,0,0,Argentina,1,...,0,1,0,0,True,False,True,False,True,False


In [11]:
train_Argentina['num_sold']

0          63.0
1          66.0
2           9.0
3          59.0
4          49.0
          ...  
136885    251.0
136886    252.0
136887     39.0
136888    233.0
136889    192.0
Name: num_sold, Length: 27390, dtype: float64

In [12]:
def train_data(train):
    delete_columns = ['id', 'date','country','store','product']
    train.drop(delete_columns, axis=1, inplace=True)
    
    y_train = train['num_sold']
    X_train = train.drop('num_sold', axis=1)
    # X_test = test.drop('num_sold', axis=1)

    from sklearn.linear_model import LogisticRegression

    clf = LogisticRegression(penalty='l2', solver='sag', random_state=0, n_jobs=-1, max_iter=1500)
    # 実行時に以下のリンクが出現した
    # https://www.kaggle.com/code/dansbecker/using-categorical-data-with-one-hot-encoding/notebook
    clf.fit(X_train, y_train)
    
    return clf


In [13]:
print(datetime.datetime.now())

2023-07-24 15:36:58.603553


In [14]:
clf_Argentina = train_data(train_Argentina.copy())

In [15]:
print(datetime.datetime.now())

2023-07-24 15:48:05.706875


In [16]:
clf_Canada = train_data(train_Canada.copy())

In [17]:
print(datetime.datetime.now())

2023-07-24 16:31:18.901311


In [18]:
clf_Estonia = train_data(train_Estonia.copy())

In [19]:
print(datetime.datetime.now())

2023-07-24 16:57:00.111214


In [20]:
clf_Japan = train_data(train_Japan.copy())

In [21]:
print(datetime.datetime.now())

2023-07-24 17:30:29.403788


In [22]:
clf_Spain = train_data(train_Spain.copy())

In [23]:
print(datetime.datetime.now())

2023-07-24 17:59:31.094624


# モデルの適用

In [24]:
X_test = test.copy().drop('num_sold', axis=1)
delete_columns = ['id', 'date','country','store','product']
X_test.drop(delete_columns, axis=1, inplace=True)

In [25]:
y_pred_Argentina = clf_Argentina.predict(X_test)
y_pred_Canada = clf_Canada.predict(X_test)
y_pred_Estonia = clf_Estonia.predict(X_test)
y_pred_Japan = clf_Japan.predict(X_test)
y_pred_Spain = clf_Spain.predict(X_test)

In [26]:
print(datetime.datetime.now())

2023-07-24 17:59:32.299478


In [27]:
test.head()

,id,date,num_sold,country_Argentina,country_Canada,country_Estonia,country_Japan,country_Spain,country,store_Kagglazon,...,thursday,friday,saturday,sunday,is_month_end,is_month_start,is_quarter_end,is_quarter_start,is_year_end,is_year_start
0,136950,2022-01-01,NaN,1,0,0,0,0,Argentina,0,...,0,0,1,0,False,True,False,True,False,True
1,136951,2022-01-01,NaN,1,0,0,0,0,Argentina,0,...,0,0,1,0,False,True,False,True,False,True
2,136952,2022-01-01,NaN,1,0,0,0,0,Argentina,0,...,0,0,1,0,False,True,False,True,False,True
3,136953,2022-01-01,NaN,1,0,0,0,0,Argentina,0,...,0,0,1,0,False,True,False,True,False,True
4,136954,2022-01-01,NaN,1,0,0,0,0,Argentina,0,...,0,0,1,0,False,True,False,True,False,True


In [28]:
 sub = test[['id','country_Argentina','country_Canada','country_Estonia','country_Japan','country_Spain']]

In [29]:
sub['num_sold_Argentina'] = list(map(int, y_pred_Argentina))
sub['num_sold_Canada'] = list(map(int, y_pred_Canada))
sub['num_sold_Estonia'] = list(map(int, y_pred_Estonia))
sub['num_sold_Japan'] = list(map(int, y_pred_Japan))
sub['num_sold_Spain'] = list(map(int, y_pred_Spain))

C:\Users\s-heya\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\s-heya\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\s-heya\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats

In [30]:
sub['num_sold'] = ""

C:\Users\s-heya\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [31]:
sub.head()

,id,country_Argentina,country_Canada,country_Estonia,country_Japan,country_Spain,num_sold_Argentina,num_sold_Canada,num_sold_Estonia,num_sold_Japan,num_sold_Spain,num_sold
0,136950,1,0,0,0,0,37,1041,501,108,153,
1,136951,1,0,0,0,0,37,772,156,108,156,
2,136952,1,0,0,0,0,5,19,12,17,14,
3,136953,1,0,0,0,0,24,268,548,108,98,
4,136954,1,0,0,0,0,17,77,46,76,51,


In [32]:
sub.loc[sub['country_Argentina'] == 1, 'num_sold'] = sub['num_sold_Argentina']
sub.loc[sub['country_Canada'] == 1, 'num_sold'] = sub['num_sold_Canada']
sub.loc[sub['country_Estonia'] == 1, 'num_sold'] = sub['num_sold_Estonia']
sub.loc[sub['country_Japan'] == 1, 'num_sold'] = sub['num_sold_Japan']
sub.loc[sub['country_Spain'] == 1, 'num_sold'] = sub['num_sold_Spain']
sub.to_csv('sub.csv')

C:\Users\s-heya\Anaconda3\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [33]:
sub_csv = sub[['id', 'num_sold']]
sub_csv.to_csv('submission.csv', index=False)
sub_csv

,id,num_sold
0,136950,37
1,136951,37
2,136952,5
3,136953,24
4,136954,17
...,...,...
27370,164320,526
27371,164321,526
27372,164322,74
27373,164323,422
